In this part we are trying to view the Parquet file. Like how the datas are arranged inside it.
Parquet files are very similar to excel file. But the differnece come in how they are stored and processed in the memory.

In [3]:
#libraries and Paths
import pandas as pd
import glob
import os


In [4]:
important_codes = ["IDRSSD", "RCFD2170","RCON2170","RCON2210","RCON2365","RCON6648","RCON2604","RCONJ474","RCONJ473","RCFN2200","RCON2200"
                  ]
codes = [ c for c in important_codes ]
#print(codes)

In [86]:
# # Read parquet file
# file_path1 = r"C:\Users\32011\Dropbox\Call reports data\Output\All Quarters\2020-03-31\RC_2020-03-31.parquet"
# file_path2 = r"C:\Users\32011\Dropbox\Call reports data\Output\All Quarters\2020-03-31\RCA_2020-03-31.parquet"
# df1 = pd.read_parquet(file_path1)
# df1=df1.loc[:, df1.columns.str.strip() != '']
# # Show basic info about the file
# print("Shape of the data:", df1.shape)
# print("\nColumn names:\n", df1.columns.tolist())
# print("\nData types:\n", df1.dtypes)

# # Show first 10 rows
# print("\nSample data:\n", df1.head(10))


# df2 = pd.read_parquet(file_path2)
# df2=df2.loc[:, df2.columns.str.strip() != '']
# # df = df.loc[:, ~df.columns.str.contains("^Unnamed")] 
# print("data shape:", df2.shape)
# print("\n column names: \n", df2.columns.tolist())
# print("\n sample data: \n", df2.head(20))

# id_col1 = df1.columns[0]
# id_col2 = df2.columns[0]
# merged = pd.merge(df1, df2, left_on=id_col1, right_on =id_col2, how = "outer")
# print("shape of the merged data:", merged.shape)


# print(merged.head(10))


In [6]:
input_folder = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. Quarters folder"
output_folder =r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\1"
os.makedirs(output_folder, exist_ok = True)
for quarter in os.listdir(input_folder):
    quarter_path = os.path.join(input_folder, quarter)
    if not os.path.isdir(quarter_path):
        continue
    print(f"Processing quarter: {quarter}")
    files = glob.glob(os.path.join(quarter_path, "*.parquet"))
    print("Found", len(files), "parquet files.")

    if not files:
        continue

    merged_df = None

    for i, file in enumerate(files, 1):
        #print(f"Reading {i}/{len(files)} -> {os.path.basename(file)}")
        
        # Read file
        df = pd.read_parquet(file)
        
        # Remove blank/unnamed columns
        df = df.loc[:, df.columns.str.strip() != ""]
        df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
        
        # Identify ID column (first column)
        id_col = df.columns[0]
        df[id_col] = pd.to_numeric(df[id_col], errors = "coerce").astype("Int64")
        
        # Merge
        if merged_df is None:
            merged_df = df
        else:
            new_cols = [col for col in df.columns if col not in merged_df.columns or col == id_col]
            merged_df = pd.merge(merged_df, df[new_cols], on=id_col, how="outer")
    df_sorted = merged_df.sort_values(by = id_col, ascending= True)
    output_file = os.path.join(output_folder, f"{quarter}.parquet")
    df_sorted.to_parquet(output_file, index= False)
    print("\nFinal merged shape:", df_sorted.shape)
# print("\nColumns:\n", df_sorted.columns.tolist())
# print("\nSample data:\n", df_sorted.head(10))


    print(f"Merged file saved to:{output_file}")


Processing quarter: 2001-03-31
Found 27 parquet files.

Final merged shape: (8857, 1424)
Merged file saved to:C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\1\2001-03-31.parquet
Processing quarter: 2001-06-30
Found 27 parquet files.

Final merged shape: (8790, 1635)
Merged file saved to:C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\1\2001-06-30.parquet
Processing quarter: 2001-09-30
Found 27 parquet files.

Final merged shape: (8758, 1635)
Merged file saved to:C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\1\2001-09-30.parquet
Processing quarter: 2001-12-31
Found 28 parquet files.

Final merged shape: (8689, 1798)
Merged file saved to:C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\1\2001-12-31.parquet
Processing quarter: 2002-03-31
Found 28 parquet files.

Final merged shape: (8605, 1833)
Merged file saved to:C:\Users\32011

In [126]:
merged_file = r"C:\Users\32011\Dropbox\Call reports data\Output\Intermediate folders\merged_parquet_all.parquet"
df = pd.read_parquet(merged_file)
# Ensure we have the IDRSSD column
id_col = df.columns[0]

# Check available codes
available_codes = [c for c in codes if c in df.columns]
missing_codes = [c for c in codes if c not in df.columns]

# Inform about missing codes
if missing_codes:
    print("⚠️ These codes were not found in the data:", missing_codes)

# Keep only IDRSSD + found codes
filtered_df = df[[id_col] + available_codes]

# Save to parquet
output_file = "filtered_codes.parquet"
filtered_df.to_parquet(output_file, index=False)

print(f"\n✅ Filtered parquet file saved as {output_file}")
print("Shape:", filtered_df.shape)
print("Columns:", filtered_df.columns.tolist())

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\32011\\Dropbox\\Call reports data\\Output\\Merged Quarters\\merged_parquet_all.parquet'

In [ ]:
# df = pd.read_parquet(merged_file)
# code = "RCON2216_y"

# if code in df.columns:
#     print(df[[df.columns[0], code]])
# else:
#     print(f"⚠️ Code '{code}' not found in merged file!")

Creating a smaller parquet file from the bigger parquet file using the important pre specified codes. Please note that not all the codes in the important codes dictionary will find their place in the final one, many codes are not there in the merged file


In [7]:
parq_folder1 = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\1"
parq_folder2 = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\2"
os.makedirs(parq_folder2, exist_ok = True)

for file in os.listdir(parq_folder1):
    if not file.endswith(".parquet"):
        continue
    merged_file = os.path.join(parq_folder1, file)
    print(f"processing: {file}")
    df = pd.read_parquet(merged_file)


    # Keep only the columns that exist in the file
    cols_to_keep = [col for col in codes if col in df.columns]
    if not cols_to_keep:
        print("No matching codes found, skipping. ")
        continue
        
    subset_df = df[cols_to_keep]

    # Save to new parquet
    output_file = os.path.join(parq_folder2, f"{file}")
    subset_df.to_parquet(output_file, index=False)


    print("Subset parquet saved to:", {output_file})
    print(f"Shape: {subset_df.shape}")
    print("Columns:", subset_df.columns.tolist())


processing: 2001-03-31.parquet
Subset parquet saved to: {'C:\\Users\\32011\\Dropbox\\Rajib\\Call reports data\\Output\\Intermediate folders\\Parq. folder\\2\\2001-03-31.parquet'}
Shape: (8857, 9)
Columns: ['IDRSSD', 'RCFD2170', 'RCON2170', 'RCON2210', 'RCON2365', 'RCON6648', 'RCON2604', 'RCFN2200', 'RCON2200']
processing: 2001-06-30.parquet
Subset parquet saved to: {'C:\\Users\\32011\\Dropbox\\Rajib\\Call reports data\\Output\\Intermediate folders\\Parq. folder\\2\\2001-06-30.parquet'}
Shape: (8790, 9)
Columns: ['IDRSSD', 'RCFD2170', 'RCON2170', 'RCON2210', 'RCON2365', 'RCON6648', 'RCON2604', 'RCFN2200', 'RCON2200']
processing: 2001-09-30.parquet
Subset parquet saved to: {'C:\\Users\\32011\\Dropbox\\Rajib\\Call reports data\\Output\\Intermediate folders\\Parq. folder\\2\\2001-09-30.parquet'}
Shape: (8758, 9)
Columns: ['IDRSSD', 'RCFD2170', 'RCON2170', 'RCON2210', 'RCON2365', 'RCON6648', 'RCON2604', 'RCFN2200', 'RCON2200']
processing: 2001-12-31.parquet
Subset parquet saved to: {'C:\\Us

Converting the parquet files to csv file 

In [8]:
parquet_files_folder = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Parq. folder\2" 
csv_output_folder = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Excel folders"
os.makedirs(csv_output_folder, exist_ok = True)

for f in os.listdir(parquet_files_folder):
    if f.endswith(".parquet"):
        parquet_path = os.path.join(parquet_files_folder, f)

        csv_name = os.path.splitext(f)[0]+ ".csv"
        csv_path = os.path.join(csv_output_folder, csv_name)
        print(f"converting {f}")
        df = pd.read_parquet(parquet_path)
        df.to_csv(csv_path, index = False)

print(f"All files converted to excel")

converting 2001-03-31.parquet
converting 2001-06-30.parquet
converting 2001-09-30.parquet
converting 2001-12-31.parquet
converting 2002-03-31.parquet
converting 2002-06-30.parquet
converting 2002-09-30.parquet
converting 2002-12-31.parquet
converting 2003-03-31.parquet
converting 2003-06-30.parquet
converting 2003-09-30.parquet
converting 2003-12-31.parquet
converting 2004-03-31.parquet
converting 2004-06-30.parquet
converting 2004-09-30.parquet
converting 2004-12-31.parquet
converting 2005-03-31.parquet
converting 2005-06-30.parquet
converting 2005-09-30.parquet
converting 2005-12-31.parquet
converting 2006-03-31.parquet
converting 2006-06-30.parquet
converting 2006-09-30.parquet
converting 2006-12-31.parquet
converting 2007-03-31.parquet
converting 2007-06-30.parquet
converting 2007-09-30.parquet
converting 2007-12-31.parquet
converting 2008-03-31.parquet
converting 2008-06-30.parquet
converting 2008-09-30.parquet
converting 2008-12-31.parquet
converting 2009-03-31.parquet
converting

In [135]:
csv_folder = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Intermediate folders\Excel folders"
por_file = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Final Output folder\Version 2\Reporter_infor_file.xlsx"
op_folder = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Final Output folder"
os.makedirs(op_folder, exist_ok = True)

dictionary_file = r"C:\Users\32011\Dropbox\Rajib\Call reports data\Output\Final Output folder\Version 2\Master_Dictionary_RC_RI_SU.xlsx"
codes_to_keep = ["Financial Institution Name"]

# Load POR file
por_df = pd.read_excel(por_file, dtype={"IDRSSD": "Int64"})
por_subset = por_df[["IDRSSD"] + [c for c in codes_to_keep if c in por_df.columns]]
dict_df = pd.read_excel(dictionary_file)
code_map = dict(zip(dict_df["Code"], dict_df["Description"]))
print("Files in csv_folder:", os.listdir(csv_folder))

for file in os.listdir(csv_folder):
    if file.endswith(".csv"): #and file not in [os.path.basename(por_file), os.path.basename(dictionary)]:
        csv_path = os.path.join(csv_folder, file)

        print(f"\n Processing {file} ...")

        # Load Excel file
        df = pd.read_csv(csv_path, dtype={"IDRSSD": "Int64"})

        # Merge with POR data (left join → keep all IDs from the Excel file)
        merged_df = pd.merge(df, por_subset, on="IDRSSD", how="left")

        renamed_cols = {col: code_map.get(col,col) for col in merged_df.columns}
        merged_df.rename(columns=renamed_cols, inplace=True)

        # Save back (overwrite or create new)
        output_path = os.path.join(op_folder, file)
        merged_df.to_csv(output_path, index=False)

        print(f"Updated file saved as {output_path}")

Files in csv_folder: ['subset_merged_2001-03-31.csv', 'subset_merged_2001-06-30.csv', 'subset_merged_2001-09-30.csv', 'subset_merged_2001-12-31.csv', 'subset_merged_2002-03-31.csv', 'subset_merged_2002-06-30.csv', 'subset_merged_2002-09-30.csv', 'subset_merged_2002-12-31.csv', 'subset_merged_2003-03-31.csv', 'subset_merged_2003-06-30.csv', 'subset_merged_2003-09-30.csv', 'subset_merged_2003-12-31.csv', 'subset_merged_2004-03-31.csv', 'subset_merged_2004-06-30.csv', 'subset_merged_2004-09-30.csv', 'subset_merged_2004-12-31.csv', 'subset_merged_2005-03-31.csv', 'subset_merged_2005-06-30.csv', 'subset_merged_2005-09-30.csv', 'subset_merged_2005-12-31.csv', 'subset_merged_2006-03-31.csv', 'subset_merged_2006-06-30.csv', 'subset_merged_2006-09-30.csv', 'subset_merged_2006-12-31.csv', 'subset_merged_2007-03-31.csv', 'subset_merged_2007-06-30.csv', 'subset_merged_2007-09-30.csv', 'subset_merged_2007-12-31.csv', 'subset_merged_2008-03-31.csv', 'subset_merged_2008-06-30.csv', 'subset_merged_200

In [112]:
print("Files found:", os.listdir(por_file))


Files found: ['2020-03-31.xlsx', 'file.csv', 'Master_Dictionary_RC_RI_SU.xlsx', 'merged_parquet_all.csv', 'Reporter_infor_file.xlsx']
